# Computational Drug Discovery Tutorial

Welcome to this hands-on notebook accompanying the blog tutorial.  
We'll cover:

1. **Environment Setup**  
2. **Protein Preparation**  
3. **Ligand Preparation**  
4. **Docking with AutoDock Vina**  
5. **Optional Mini Virtual Screening**  
6. **Basic ML Model**  
7. **Next Steps & Wrap-Up**  

---

## 1. Environment & Initial Setup



In [13]:
import sys
import os
import rdkit
import deepchem
from rdkit import Chem
print("RDKit version:", rdkit.__version__)
print("DeepChem version:", deepchem.__version__)

# Define a data directory (adjust to your structure)
DATA_DIR = "./data"
RESULTS_DIR = "./results"

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

print("Data directory:", DATA_DIR)
print("Results directory:", RESULTS_DIR)


RDKit version: 2022.09.5
DeepChem version: 2.8.0
Data directory: ./data
Results directory: ./results


In [14]:
import os
import requests

DATA_DIR = "./data"
os.makedirs(DATA_DIR, exist_ok=True)

# 1) Download the 2R3J PDB directly from RCSB
pdb_id = "2R3J"
pdb_url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
pdb_file_path = os.path.join(DATA_DIR, f"{pdb_id}.pdb")

print(f"Downloading PDB {pdb_id} to {pdb_file_path} ...")
response = requests.get(pdb_url)
with open(pdb_file_path, "wb") as f:
    f.write(response.content)
print("Download complete!")

# 2) Define some known CDK2 inhibitors by SMILES
#    (These can be used in the ligand preparation step)
ligand_smiles_dict = {
    "Roscovitine": "CC(C)c1ccc(cc1NC(=O)C(CC)NC(=O)c1ccc(cc1)C#N)N(C)C",
}

print("We have the following ligands to work with:")
for lig_name, smiles in ligand_smiles_dict.items():
    print(f"  - {lig_name}: {smiles}")


Download complete!
We have the following ligands to work with:
  - Roscovitine: CC(C)c1ccc(cc1NC(=O)C(CC)NC(=O)c1ccc(cc1)C#N)N(C)C


In [15]:
import os
import subprocess
from rdkit import Chem

def prepare_protein(input_pdb, output_pdb):
    """
    Remove water molecules and add hydrogens using Open Babel,
    then save the cleaned structure to output_pdb.
    """
    no_water_pdb = output_pdb.replace(".pdb", "_noWater.pdb")
    
    # 1) Remove water (-xr option in Open Babel)
    cmd_remove_water = f"obabel {input_pdb} -O {no_water_pdb} -xr"
    subprocess.run(cmd_remove_water, shell=True, check=True)
    
    # 2) Add hydrogens (-h)
    cmd_add_h = f"obabel {no_water_pdb} -O {output_pdb} -h"
    subprocess.run(cmd_add_h, shell=True, check=True)
    
    print(f"Protein prepared. Final file: {output_pdb}")

# --- Usage Example ---

pdb_id = "2R3J"
pdb_file_path = os.path.join(DATA_DIR, f"{pdb_id}.pdb")
cleaned_pdb_path = os.path.join(DATA_DIR, f"{pdb_id}_cleaned.pdb")

print("Preparing protein...")
prepare_protein(pdb_file_path, cleaned_pdb_path)

# Optional: Load the cleaned file into RDKit to confirm it can parse properly
protein_mol = Chem.MolFromPDBFile(cleaned_pdb_path, removeHs=False)
if protein_mol:
    print("RDKit loaded cleaned protein successfully.")
    print("Number of atoms:", protein_mol.GetNumAtoms())
else:
    print("Warning: RDKit could not parse the cleaned protein.")


Preparing protein...


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is ./data/2R3J.pdb)

1 molecule converted


Protein prepared. Final file: ./data/2R3J_cleaned.pdb


1 molecule converted
[11:46:20] Explicit valence for atom # 241 C, 5, is greater than permitted


In [16]:
import os
from rdkit import Chem
from rdkit.Chem import AllChem

def prepare_ligand(smiles: str, name: str, outdir: str, 
                   embed_seed: int = 42) -> str:
    """
    Convert SMILES to 3D with RDKit, add hydrogens, and do
    a quick force-field minimization. Return the final .pdb path.
    """
    # Create RDKit Mol
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"ERROR: Could not parse SMILES for ligand {name}.")
        return ""

    # Add hydrogens
    mol = Chem.AddHs(mol)

    # 3D embedding
    status = AllChem.EmbedMolecule(mol, randomSeed=embed_seed)
    if status != 0:
        print(f"WARNING: Embedding failed for {name} (status={status}).")
    else:
        # UFF or MMFF Minimization
        AllChem.UFFOptimizeMolecule(mol, maxIters=1000)
        # Alternatively: 
        # AllChem.MMFFOptimizeMolecule(mol, maxIters=1000)

    # Write out as .pdb (could also do .sdf)
    outfile = os.path.join(outdir, f"{name}.pdb")
    Chem.MolToPDBFile(mol, outfile)
    return outfile

# Example usage
os.makedirs(DATA_DIR, exist_ok=True)

for lig_name, smi in ligand_smiles_dict.items():
    pdb_path = prepare_ligand(smi, lig_name, DATA_DIR)

    if pdb_path:
        print(f"{lig_name} prepared and saved at: {pdb_path}")


Roscovitine prepared and saved at: ./data/Roscovitine.pdb
